# 1.2 视图和Url配置

视图是动态网页的基础之一,它用来定义每个 http 请求所对应的反馈,用装饰器`@app.route(url)`来绑定url,再在它下面写对应的view.
与django不同,flask的视图和url是结合在一起的.这样的好处是写起来比较直观,当然弊端也有,就是一旦程序大了起来会比较难维护.
事实上我们上一节已经构建了helloworld.我们来仔细研究下:


In [1]:
!cat ../codes/firstapp/manager.py

#coding:UTF-8
import os
import sys

root = os.path.dirname(__file__)

sys.path.insert(0, os.path.join(root, 'site-packages'))

from firstappplus import manager

if __name__ == '__main__':
    manager.run()

In [2]:
!cat ../codes/firstapp/firstappplus.py

#coding:UTF-8
from flask import Flask
from flask.ext.script import Manager

app = Flask(__name__)
app.debug = True
manager = Manager(app)

@app.route('/')
def hello():
    return "Hello, world! - flask"

@app.route('/<name>')
def name(name):
    return "Hello, world! - {name}".format(name=name)

在第二个视图中我们看到`<name>`这样一个动态地址,它表示我们的视图装饰器会捕捉在根地址后面的内容作为一个变量,然后把变量作为后面注册的视图方法的参数.

可以试试运行这个app,然后打开网页看看效果.

> 我们来练习下自己写个view

In [7]:
%%writefile ../codes/helloworld/helloworld.py

#coding:UTF-8
from flask import Flask,request# 加了request来获取浏览器发来的信息
from flask.ext.script import Manager

app = Flask(__name__)
app.debug = True
manager = Manager(app)

@app.route('/')
def hello():
    return "Hello, world! - flask"

@app.route('/<name>')
def name(name):
    return "Hello, world! - {name}".format(name=name)

@app.route("/info")
def info():
    user_agent = request.headers.get('User-Agent')
    return '<p>Your browser is {agent}</p>'.format(agent=user_agent)


Overwriting ../codes/helloworld/helloworld.py


In [5]:
%%writefile ../codes/helloworld/manager.py
#coding:UTF-8
import os
import sys

root = os.path.dirname(__file__)

sys.path.insert(0, os.path.join(root, 'site-packages'))

from helloworld import manager

if __name__ == '__main__':
    manager.run()

Writing ../codes/helloworld/manager.py


In [8]:
!python ../codes/helloworld/manager.py runserver

 * Restarting with stat
 * Debugger is active!
 * Debugger pin code: 673-134-554
127.0.0.1 - - [19/Jan/2016 13:33:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 13:33:21] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 13:33:30] "GET /flask HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 13:33:30] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 13:33:43] "GET /myname HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 13:33:43] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 13:33:56] "GET /info HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 13:33:56] "GET /favicon.ico HTTP/1.1" 200 -
^C


## 请求调度

程序收到客户端发来的请求时,要找到处理该请求的视图函数。为了完成这个任务,Flask 会在程序的 URL 映射中查找请求的 URL。URL 映射是 URL 和视图函数之间的对应关系。 Flask 使用 app.route 修饰器或者非修饰器形式的 app.add_url_rule() 生成映射。

要想查看 Flask 程序中的 URL 映射是什么样子,我们可以在 Python shell 中检查为 helloworld.py 生成的映射:

    $python manage.py shell
    >>> from myapp import app
    >>> app.url_map
    Map([<Rule '/info' (HEAD, OPTIONS, GET) -> info>,
     <Rule '/' (HEAD, OPTIONS, GET) -> hello>,
     <Rule '/static/<filename>' (HEAD, OPTIONS, GET) -> static>,
     <Rule '/<name>' (HEAD, OPTIONS, GET) -> name>])
     
     
这是url和view的映射表,各项分别是<url,方法,view>
其中`<Rule '/static/<filename>' (HEAD, OPTIONS, GET) -> static>`,不是我们定义的,是flask默认的静态文件映射.




## 上下文全局变量

在flask中有个机制叫上下文,用来封装一些变量,上下文变量都是全局变量,只是每次程序有了变化它里面的值会根据变化改变.

Flask上下文全局变量:

变量|说明
---|---
current_app |当前激活程序的程序实例
g |处理请求时用作临时存储的对象。每次请求都会重设这个变量
request |请求对象,封装了客户端发出的 HTTP 请求中的内容
session |用户会话,用于存储请求之间需要“记住”的值的词典

之前我们已经用过request了,看代码一定觉得它写的不科学,因为对于不同的请求,它的request肯定都是不一样的,怎么会是全局变量呢

事实上request只是在一个线程上是全局变量,每个线程上的request都不同,不会相互影响,要不也不叫上下文了

## 请求钩子

有时在处理请求之前或之后执行代码会很有用。例如,在请求开始时,我们可能需要创 建数据库连接或者认证发起请求的用户。
为了避免在每个视图函数中都使用重复的代码, Flask 提供了注册通用函数的功能,注册的函数可在请求被分发到视图函数之前或之后 调用。
请求钩子使用修饰器实现。Flask 支持以下 4 种钩子。

请求钩子|说明
---|---
before_first_request|注册一个函数,在处理第一个请求之前运行。
before\_request|注册一个函数,在每次请求之前运行。
after\_request|注册一个函数,如果没有未处理的异常抛出,在每次请求之后运行。
teardown\_request|注册一个函数,即使有未处理的异常抛出,也在每次请求之后运行。

在请求钩子函数和视图函数之间共享数据一般使用上下文全局变量 g。
例如,before\_request 处理程序可以从数据库中加载已登录用户,并将其保存到 g.user 中。
随后调用视 图函数时,视图函数再使用 g.user 获取用户。

> 写个例子看看before\_request和g是如何使用的

In [17]:
%%writefile ../codes/helloworld/helloworld.py
#coding:UTF-8
from flask import Flask,request,g
from flask.ext.script import Manager
import time

app = Flask(__name__)
app.debug = True
manager = Manager(app)

#新增
@app.before_request
def before_request():
        g.time = time.asctime()


@app.route('/')
def hello():
    return "time:{count} Hello, world! - flask".format(count=g.time)

@app.route('/<name>')
def name(name):
    return "time:{count} Hello, world! - {name}".format(count=g.time,name=name)

@app.route("/info")
def info():
    user_agent = request.headers.get('User-Agent')
    return '<p>time:{count}</p><p>Your browser is {agent}</p>'.format(count=g.time,agent=user_agent)



Overwriting ../codes/helloworld/helloworld.py


In [12]:
!python ../codes/helloworld/manager.py runserver

 * Restarting with stat
 * Debugger is active!
 * Debugger pin code: 673-134-554
127.0.0.1 - - [19/Jan/2016 13:40:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 13:40:26] "GET /favicon.ico HTTP/1.1" 200 -
^C


这个例子会在每个请求回应之前计算出当前时间,然后存在g里面在请求中显示出来.

## 响应

从url接受请求后会找到对应的view处理请求,处理完后返回一个响应,可以看到返回的响应是字符串,
其实这是简略的写法,一个完整的响应应该是

    return "字符串",400,{"a":"af"}

也就是一个由字符串(可以使html格式),状态码(默认400),http头组(可选)成的元组

### Response 对象

flask可以返回一个response对象用来调用一些额外的方法,比如设置cookie,构造一个response对象
可以使用`make_response()`函数来构造,它的参数和上面响应是一样的,也可以只传入字符串

#### cookie

before_request常用来作cookie验证,flask中设置和读取cookie很简单

修改index:

In [18]:
%%writefile ../codes/helloworld/helloworld.py
#coding:UTF-8
from flask import Flask,request,g,make_response
from flask.ext.script import Manager
import time

app = Flask(__name__)
app.debug = True
manager = Manager(app)

@app.before_request
def before_request():
        g.time = time.asctime()
        ##修改
        if request.cookies.get('answer'):
            g.answer = request.cookies.get('answer')
        else:
            g.answer = "0"


@app.route('/')
def hello():
    ##修改
    response = make_response("""<p>time:{count}</p>
        <h1>Hello, world! - flask</h1>
        <p>this document carries a cookie </p>
        <p>answer={answer}</p>""".format(count=g.time,answer=g.answer))
    response.set_cookie('answer', '42')
    return response

@app.route('/<name>')
def name(name):
    return "time:{count} Hello, world! - {name}".format(count=g.time,name=name)

@app.route("/info")
def info():
    user_agent = request.headers.get('User-Agent')
    return '<p>time:{count}</p><p>Your browser is {agent}</p>'.format(count=g.time,agent=user_agent)


Overwriting ../codes/helloworld/helloworld.py


In [16]:
!python ../codes/helloworld/manager.py runserver

 * Restarting with stat
 * Debugger is active!
 * Debugger pin code: 673-134-554
127.0.0.1 - - [19/Jan/2016 13:46:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 13:46:27] "GET /favicon.ico HTTP/1.1" 200 -
^C


### 错误

除了正确地响应,当然也可能有错误的响应,也就是HttpError,什么404之类的

返回一个错误一般用函数`abort()`来实现,参数为错误类型

我们新增一个view叫`login`因为现在没做呢 ,我们希望用户点进去的是后报404错误,那么这样写

In [21]:
%%writefile ../codes/helloworld/helloworld.py
#coding:UTF-8
from flask import Flask,request,g,make_response,abort
from flask.ext.script import Manager
import time

app = Flask(__name__)
app.debug = True
manager = Manager(app)

@app.before_request
def before_request():
        g.time = time.asctime()
        if request.cookies.get('answer'):
            g.answer = request.cookies.get('answer')
        else:
            g.answer = "0"


@app.route('/')
def hello():
    response = make_response("""<p>time:{count}</p>
        <h1>Hello, world! - flask</h1>
        <p>this document carries a cookie </p>
        <p>answer={answer}</p>""".format(count=g.time,answer=g.answer))
    response.set_cookie('answer', '42')
    return response

@app.route('/<name>')
def name(name):
    return "time:{count} Hello, world! - {name}".format(count=g.time,name=name)

@app.route("/info")
def info():
    user_agent = request.headers.get('User-Agent')
    return '<p>time:{count}</p><p>Your browser is {agent}</p>'.format(count=g.time,agent=user_agent)

@app.route('/login')
def login():
    abort(404)


Overwriting ../codes/helloworld/helloworld.py


In [22]:
!python ../codes/helloworld/manager.py runserver

 * Restarting with stat
 * Debugger is active!
 * Debugger pin code: 673-134-554
127.0.0.1 - - [19/Jan/2016 13:59:01] "GET /login HTTP/1.1" 404 -
127.0.0.1 - - [19/Jan/2016 13:59:01] "GET /favicon.ico HTTP/1.1" 200 -
^C


实际上abort是怎么工作的呢?其实它只是返回了flask定制好的错误模板,如果想自己使用自己定义的错误模板呢可以这样:


In [25]:
%%writefile ../codes/helloworld/helloworld.py
#coding:UTF-8
from flask import Flask,request,g,make_response,abort
from flask.ext.script import Manager
import time

app = Flask(__name__)
app.debug = True
manager = Manager(app)

@app.before_request
def before_request():
        g.time = time.asctime()
        if request.cookies.get('answer'):
            g.answer = request.cookies.get('answer')
        else:
            g.answer = "0"
##新增
@app.errorhandler(404)
def page_not_found(error):
    return "404,page not found!",404,{"a":"af"}
@app.route('/')
def hello():
    response = make_response("""<p>time:{count}</p>
        <h1>Hello, world! - flask</h1>
        <p>this document carries a cookie </p>
        <p>answer={answer}</p>""".format(count=g.time,answer=g.answer))
    response.set_cookie('answer', '42')
    return response

@app.route('/<name>')
def name(name):
    return "time:{count} Hello, world! - {name}".format(count=g.time,name=name)

@app.route("/info")
def info():
    user_agent = request.headers.get('User-Agent')
    return '<p>time:{count}</p><p>Your browser is {agent}</p>'.format(count=g.time,agent=user_agent)

@app.route('/login')
def login():
    abort(404)



Overwriting ../codes/helloworld/helloworld.py


In [24]:
!python ../codes/helloworld/manager.py runserver

 * Restarting with stat
 * Debugger is active!
 * Debugger pin code: 673-134-554
127.0.0.1 - - [19/Jan/2016 14:05:46] "GET /login HTTP/1.1" 404 -
127.0.0.1 - - [19/Jan/2016 14:05:47] "GET /favicon.ico HTTP/1.1" 200 -
^C


错误是可以自定义页面的,后面会介绍如何使用模板,这个技术一样也可以用在自定错误页面上

### 重新定向

最有还有一种特殊的响应,就是重新定向了,它的作用是告诉浏览器一个新地址,用来加载页面,有点像推卸责任的感觉

重新定向用函数`redirect()`实现,传入参数为url字符串,而url_for可以配合使用获取某个绑定url指向的view

新增一个`infos`让它重新定向到`info`:

In [26]:
%%writefile ../codes/helloworld/helloworld.py
#coding:UTF-8
from flask import Flask,request,g,make_response,abort,redirect,url_for
from flask.ext.script import Manager
import time

app = Flask(__name__)
app.debug = True
manager = Manager(app)

@app.before_request
def before_request():
        g.time = time.asctime()
        if request.cookies.get('answer'):
            g.answer = request.cookies.get('answer')
        else:
            g.answer = "0"
##新增
@app.errorhandler(404)
def page_not_found(error):
    return "404,page not found!",404,{"a":"af"}
@app.route('/')
def hello():
    response = make_response("""<p>time:{count}</p>
        <h1>Hello, world! - flask</h1>
        <p>this document carries a cookie </p>
        <p>answer={answer}</p>""".format(count=g.time,answer=g.answer))
    response.set_cookie('answer', '42')
    return response

@app.route('/<name>')
def name(name):
    return "time:{count} Hello, world! - {name}".format(count=g.time,name=name)

@app.route("/info")
def info():
    user_agent = request.headers.get('User-Agent')
    return '<p>time:{count}</p><p>Your browser is {agent}</p>'.format(count=g.time,agent=user_agent)

@app.route('/login')
def login():
    abort(404)
@app.route('/infos')
def infos():
    return redirect(url_for('info'))

Overwriting ../codes/helloworld/helloworld.py


In [27]:
!python ../codes/helloworld/manager.py runserver

 * Restarting with stat
 * Debugger is active!
 * Debugger pin code: 673-134-554
127.0.0.1 - - [19/Jan/2016 14:10:00] "GET /info HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 14:10:00] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 14:10:05] "GET /infos HTTP/1.1" 302 -
127.0.0.1 - - [19/Jan/2016 14:10:05] "GET /info HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 14:10:05] "GET /favicon.ico HTTP/1.1" 200 -
^C


## 总结

看得出来flask的views和urls比django的直观好维护,但缺点也在这里,如果有大量的view和url维护起来可能会比较棘手.